In [1]:
import requests, re
import tiktoken
from pathlib import Path
from typing import Any

In [2]:
def f123():
    yield 1
    yield 2
    yield 3

for item in f123():
    print(item)

1
2
3


In [ ]:
from db.database import Base
from sqlalchemy import Column, Integer, String, TIMESTAMP, text
from sqlalchemy.sql import func

class Book(Base):
    __tablename__ = "metaDataBook"

    id = Column(Integer,primary_key=True,nullable=False)
    title = Column(String,nullable=False)
    authors = Column(String, nullable=False)        # TODO: add FK to author
    slug_key = Column(String, nullable=False)
    lang = Column(String, nullable=False)
    # content = Column(String,nullable=False)
    created_at = Column(TIMESTAMP(timezone=True), server_default=func.now())


In [6]:
defaults = {"title": "Book A", "lang": "en"}
overrides = {"lang": "da"}    # we want to override one key

# merged = defaults | overrides
# merged = Book(**{**defaults})
merged = defaults | overrides
obj = Book(**merged)
print(merged)

{'title': 'Book A', 'lang': 'da'}


In [8]:
from pydantic import field_validator, BaseModel

class GBBookMeta(BaseModel):
    authors:list[dict]
    @field_validator("authors", mode="before")
    def clear_nulls(cls, authors:list[dict]):
        d = [{k:v if v else None for k,v in author.items() } for author in authors ]
        return d

gb = GBBookMeta(authors=[{"name": "Jens", "birth_year": "dsa"}])
gb

GBBookMeta(authors=[{'name': 'Jens', 'birth_year': 'dsa'}])

In [ ]:
formats = {
    "text/html": "https://www.gutenberg.org/ebooks/4145.html.images",
    "application/epub+zip": "https://www.gutenberg.org/ebooks/4145.epub3.images",
    "application/x-mobipocket-ebook": "https://www.gutenberg.org/ebooks/4145.kf8.images",
    "text/plain; charset=us-ascii": "https://www.gutenberg.org/ebooks/4145.txt.utf-8",
    "application/rdf+xml": "https://www.gutenberg.org/ebooks/4145.rdf",
    "image/jpeg": "https://www.gutenberg.org/cache/epub/4145/pg4145.cover.medium.jpg",
    "application/octet-stream": "https://www.gutenberg.org/files/4145/4145-h.zip"
}

any("text/plain;" in k for k in formats.keys()) 

False

In [ ]:
from azure.search.documents.indexes.models import (SimpleField, SearchFieldDataType)
from pydantic import Field, BaseModel

index_fields= [
            SimpleField(name="uuid_str", type=SearchFieldDataType.String, key=True),
            SimpleField(name="chunk_id", type=SearchFieldDataType.Int32, filterable=True)
        ]

class SearchItem(BaseModel):
    uuid_str: str|None = Field(default=None)
    chunk_id: int|None = Field(default=None)

In [12]:
SearchItem.model_fields.keys()

dict_keys(['uuid_str', 'chunk_id', 'book_name'])

In [ ]:
all(sf.name == SearchItem.model_fields.keys() for sf in index_fields)

True

In [2]:
def fetch_book(*, book_download_url="https://www.gutenberg.org/cache/epub/2701/pg2701.txt") -> str:
    """
    Fetches book. Default url is gutenberg book link to Moby Dick
    """
    r = requests.get(book_download_url, timeout=60)
    r.raise_for_status()
    return r.text

In [3]:
def strip_gutenberg_header_footer(*, book:str) -> str:
    start = re.search(r"^CHAPTER 1\.", book, re.M)
    end = re.search(r"End of the Project Gutenberg EBook of", book)

    book = book[start.start(): end.start()] if start and end else book
    return book.strip()

In [6]:
def download_or_load_from_cache(*, book_path:Path) -> str:
    if book_path.exists():
        with open(book_path, 'r', encoding="utf-8") as f:
            print(f"Loaded: {book_path.name}")
            moby_book = f.read()
        
    else:
        moby_book = fetch_book()
        moby_book = strip_gutenberg_header_footer(book=moby_book)

        with open(book_path, "w", encoding="utf-8") as f:
            f.write(moby_book)

    return moby_book

In [ ]:
from bs4.element import AttributeValueList
from bs4 import BeautifulSoup, ResultSet, Tag
from pathlib import Path

s = r"books\little-women-or-meg-jo-beth-and-amy_alcott-louisa-may_37106_en.html" #r"books\beowulf-an-anglo-saxon-epic-poem_16328_en.html"
bp = Path(s)

with open(bp, 'r', encoding="utf-8") as f:
    bs = BeautifulSoup(f)

In [2]:
href_tags = bs.find_all(href=True)
# chap_hrefs = [href for href in href_tags if href.]

In [22]:
def is_valid_href(href: str) -> bool:
    
    if href.startswith(("http://", "https://")):        # Ignore if it starts with http(s)       
        return False
    blocked_exts = (".jpg", ".jpeg", ".png", ".gif", ".pdf", ".css", ".js", ".html")        # Ignore common file extensions

    if href.lower().endswith(blocked_exts):
        return False
    return True

In [ ]:
href_tags[1]['href']#.attrs['href']).startswith('http')

'http://purl.org/dc/elements/1.1/'

In [15]:
import tiktoken
from tiktoken import Encoding
from constants import MAX_TOKENS
from chunking import fixed_size_chunks
from load_book import download_or_load_from_cache

In [4]:
def _count_tokens(text: str, enc:Encoding) -> int:
    return len(enc.encode(text))

def _batch_texts_by_tokens(texts: list[str],
                           max_tokens_per_request: int=MAX_TOKENS) -> list[list[str]]:
    """
    Greedily packs texts into batches so that the sum of tokens per batch
    stays under max_tokens_per_request.
    """
    enc = tiktoken.get_encoding("cl100k_base")
    all_batches, current_batch, token_sum_current_batch = [], [], 0
    for t in texts:
        token_len = _count_tokens(t, enc)
        if current_batch and token_sum_current_batch + token_len > max_tokens_per_request:
            all_batches.append(current_batch)
            current_batch = [t]
            token_sum_current_batch = token_len
        else:
            current_batch.append(t)
            token_sum_current_batch += token_len
    
    if current_batch:
        all_batches.append(current_batch)

    return all_batches

In [18]:
import backoff
from data_classes.vector_db import EmbeddingVec
from constants import EmbeddingDimension, MAX_TOKENS, OVERLAP
from openai import AzureOpenAI
from openai._exceptions import RateLimitError

@backoff.on_exception(wait_gen=backoff.expo, exception=RateLimitError, max_time=120, max_tries=6)
def _create_embeddings(*, embed_client:AzureOpenAI, model_deployed:str, batches:list[str]) -> list[EmbeddingVec]:
    resp = embed_client.embeddings.create(
        input=batches,
        model=model_deployed,
    )
    return [EmbeddingVec(vector=emb_obj.embedding, dim=EmbeddingDimension.SMALL) for emb_obj in resp.data]


def create_embeddings_retry(*, embed_client:AzureOpenAI, model_deployed:str, texts:list[str]) -> list[EmbeddingVec]:
    _create_embeddings(embed_client=embed_client, model_deployed=model_deployed, batches=texts)

In [ ]:
import time
from openai import RateLimitError
from data_classes.vector_db import EmbeddingVec
from pyrate_limiter import Duration, Rate, Limiter, BucketFullException
from constants import REQUESTS_PR_MIN, TOKEN_PR_MIN

# --- Limits based on your Azure deployment ---
REQ_LIMIT = Rate(REQUESTS_PR_MIN, Duration.MINUTE)              # 3,000 requests per minute
TOK_LIMIT = Rate(TOKEN_PR_MIN, Duration.MINUTE)                 # 501,000 tokens per minute

req_limiter = Limiter(REQ_LIMIT)
tok_limiter = Limiter(TOK_LIMIT)

def acquire_budget(tokens_needed: int, identity: str = "embeddings"):
    """Block until both request and token budgets allow the call."""
    while True:
        try:
            tok_limiter.try_acquire(f"{identity}_tpm", weight=tokens_needed)        # weight is amount to minus from 'total' tries amount
            req_limiter.try_acquire(f"{identity}_rpm", weight=1)
            return  # allowed
        except BucketFullException as ex:
            sleep_interval = float(ex.rate.interval)
            print(f"Exceeding budget - sleeping {sleep_interval}")
            time.sleep(sleep_interval)     # Sleep exactly as long as the limiter suggests
            # time.sleep(float(ex.meta_info['rate'].))


def limiter_create_embeddings(*, embed_client, model_deployed: str, inp_batches: list[list[str]]) -> list[EmbeddingVec]:
    """Call Azure embeddings with built-in rate limiting and graceful backoff."""
    all_embeddings = []
    enc_ = tiktoken.get_encoding("cl100k_base")

    for batch in inp_batches:
        tokens_needed = sum([_count_tokens(chunk, enc=enc_) for chunk in batch])
        print(f'tokens needed from limiter: {tokens_needed}')
        acquire_budget(tokens_needed)              # proactive pacing
        # try:
        embs = _create_embeddings(embed_client=embed_client, model_deployed=model_deployed, batches=batch)
        all_embeddings.extend(embs)
        # except RateLimitError as e:
        #     # reactive fallback if Azure throttles anyway
        #     ra = _retry_after_seconds(e)
        #     if ra:
        #         time.sleep(ra)
        #     else:
        #         time.sleep(2)
        #     # retry once after sleeping
        #     resp = embed_client.embeddings.create(input=batch, model=model_deployed)
        #     all_embeddings.extend(e.embedding for e in resp.data)
    return all_embeddings

In [ ]:
import re
from pathlib import Path

book_p = r"C:\Users\alext\Documents\Code\RAG\mobyRag\books\franken-stein.txt"
with open(Path(book_p), 'r', encoding="utf-8") as f:
    txt = f.read()

start_idx = re.search(pattern=r'\*\*\*\s?START OF TH(IS|E) PROJECT GUTENBERG EBOOK.', string=txt)
end_idx = re.search(pattern=r'\*\*\* end of the project gutenberg ebook', string=txt.lower())

s = txt[start_idx.end():end_idx.start()]
s[:50]

In [25]:
book = download_or_load_from_cache(book_key='frankenstein-or-the-modern-prometheus_shelley-mary-wollstonecraft_84_en',
                                url='https://www.gutenberg.org/cache/epub/84/pg84.txt' )

chunks = fixed_size_chunks(text=book)


 From file: frankenstein-or-the-modern-prometheus_shelley-mary-wollstonecraft_84_en.txt


In [26]:
all_batches =_batch_texts_by_tokens(texts=chunks, max_tokens_per_request=MAX_TOKENS)
# [[len(b_) for b_ in b] for b in all_batches]
len(all_batches[0][1])

7984

In [27]:
from settings import get_settings

sett = get_settings()
emb_client = AzureOpenAI(azure_endpoint=sett.AZ_OPENAI_EMBED_ENDPOINT,
                            api_version="2024-12-01-preview",
                            api_key=sett.AZ_OPENAI_EMBED_KEY)


limiter_create_embeddings(embed_client=emb_client, model_deployed=sett.EMBED_MODEL_DEPOYED, inp_batches=all_batches)

tokens needed from limiter: 7460
tokens needed from limiter: 7276
tokens needed from limiter: 7339
tokens needed from limiter: 7558
tokens needed from limiter: 7565
tokens needed from limiter: 7669
tokens needed from limiter: 7228
tokens needed from limiter: 7442
tokens needed from limiter: 7500
tokens needed from limiter: 7528
tokens needed from limiter: 7482
tokens needed from limiter: 7544
tokens needed from limiter: 7463
tokens needed from limiter: 5987


[EmbeddingVec(vector=[0.039394550025463104, 0.001639129244722426, 0.029026102274656296, 0.012198171578347683, -0.025782497599720955, 0.00026033740141429007, -0.0048446147702634335, -0.009758536703884602, -0.014526912942528725, 0.008316935040056705, 0.05134321376681328, -0.025075558573007584, -0.04798871651291847, 0.0016867783851921558, 0.044551048427820206, 0.017590317875146866, -0.014111066237092018, 0.003995594102889299, 0.023010186851024628, 0.02241414040327072, -0.003985198214650154, -0.02927561104297638, -0.047351084649562836, -0.0063485936261713505, -0.0919298529624939, 0.012045694515109062, -0.0021433434449136257, 0.04075298085808754, 0.050206564366817474, -0.01136647816747427, 0.006778302136808634, -0.023647818714380264, -0.017465563490986824, 0.01761804148554802, 0.015677422285079956, -0.009557544253766537, -0.0076030646450817585, -0.016800208017230034, -0.02847164124250412, -0.041889630258083344, 0.014249682426452637, -0.07368804514408112, 0.04247181490063667, 0.0167724862694

In [20]:
# s = str(href_tags[1].attrs['href']).startswith('http')

tags_with_valid_href = [
    tag for tag in bs.find_all(href=True) if is_valid_href(str(tag["href"]))
]

for tag in tags_with_valid_href:
    print(tag)#tag.name, tag["href"])

NameError: name 'bs' is not defined

In [ ]:
#chapters = b.find_all(lambda tag: tag.has_attr("chapter"))
chapters:ResultSet[Tag] = b.find_all(attrs={"class": "chapter"})

In [28]:
inner_html = "".join(str(x) for x in ch.contents)
inner_html

'\n<h2><a id="chap12"></a>CHAPTER XII.<br/>\n\nAlice’s Evidence</h2>\n<p>\n\n“Here!” cried Alice, quite forgetting in the flurry of the moment how large she\n\nhad grown in the last few minutes, and she jumped up in such a hurry that she\n\ntipped over the jury-box with the edge of her skirt, upsetting all the jurymen\n\non to the heads of the crowd below, and there they lay sprawling about,\n\nreminding her very much of a globe of goldfish she had accidentally upset the\n\nweek before.\n\n</p>\n<p>\n\n“Oh, I <i>beg</i> your pardon!” she exclaimed in a tone of great dismay, and\n\nbegan picking them up again as quickly as she could, for the accident of the\n\ngoldfish kept running in her head, and she had a vague sort of idea that they\n\nmust be collected at once and put back into the jury-box, or they would die.\n\n</p>\n<p>\n\n“The trial cannot proceed,” said the King in a very grave voice, “until all the\n\njurymen are back in their proper places—<i>all</i>,” he repeated with great

In [17]:
chaps = b.find_all("div", attrs="chapter")
type(chaps)

bs4.element.ResultSet

In [7]:
moby_book_p = Path("books", "moby.txt")
moby_book = download_or_load_from_cache(book_path=moby_book_p)

Loaded: moby.txt


In [1]:
import os

os.getenv("AZURE_SEARCH_ENDPOINT"), os.environ["AZURE_SEARCH_KEY"]

('https://ai-search-free-plan.search.windows.net#https://ai-search-bax.search.windows.net',
 'XEdNAWPfLCJSExkpOua9n4FiV1i5Ubpshc0WSRZXgRAzSeCIf8PD')

In [ ]:
def extract_chapters(*, book_txt: str) -> dict[int, dict[str, str]]:
    """
    Extracts chapters from a book string. Exptects chapters in format, e.g. : CHAPTER 3. <chapter title> \n <content...>
    NB - Overwrites the initial "empty" chapters from table of contents, with the actual chapter content as the book is iterated over.

    Returns a dictionary of the form:
    {
        chapter_number: {
            "title": <chapter title>,
            "content": <chapter content>
        }
    }
    """
    
    # Regex to capture chapter headings 
    pattern = re.compile(r"^CHAPTER\s+(\d+)\.?\s*([^\n]*)", re.MULTILINE)
    matches = list(pattern.finditer(book_txt))
    
    chapters = {}
    
    for chapter_i, match in enumerate(matches):
        chapter_num = int(match.group(1))
        chapter_title = match.group(2).strip()
        
        start_idx = match.end()                                                                     
        end_idx = matches[chapter_i+1].start() if chapter_i+1 < len(matches) else len(book_txt)     # End index is the start of next chapter or the end of text
        
        content = book_txt[start_idx:end_idx].strip()
        
        chapters[chapter_num] = {                                   
            "title": chapter_title,
            "content": content
        }
    
    return chapters

In [11]:
chapters = extract_chapters(book_txt=moby_book)
chapters

{1: {'title': 'Loomings.',
  'content': 'Call me Ishmael. Some years ago—never mind how long precisely—having\n\nlittle or no money in my purse, and nothing particular to interest me\n\non shore, I thought I would sail about a little and see the watery part\n\nof the world. It is a way I have of driving off the spleen and\n\nregulating the circulation. Whenever I find myself growing grim about\n\nthe mouth; whenever it is a damp, drizzly November in my soul; whenever\n\nI find myself involuntarily pausing before coffin warehouses, and\n\nbringing up the rear of every funeral I meet; and especially whenever\n\nmy hypos get such an upper hand of me, that it requires a strong moral\n\nprinciple to prevent me from deliberately stepping into the street, and\n\nmethodically knocking people’s hats off—then, I account it high time to\n\nget to sea as soon as I can. This is my substitute for pistol and ball.\n\nWith a philosophical flourish Cato throws himself upon his sword; I\n\nquietly take 

In [ ]:
def tiktoken_chunks(*, txt:str, max_tokens=600, overlap=60, encoding="cl100k_base"):
    enc = tiktoken.get_encoding(encoding)
    token_ids = enc.encode(txt)
    step = max_tokens - overlap
    out = []

    for i in range(0, len(token_ids), step):
        out.append(enc.decode(token_ids[i:i+max_tokens]))
    
    return out, token_ids

In [22]:
moby_out, token_ids = tiktoken_chunks(txt=chapters[1]['content'])

In [23]:
token_ids

[7368,
 757,
 57704,
 1764,
 301,
 13,
 4427,
 1667,
 4227,
 2345,
 37593,
 4059,
 1268,
 1317,
 24559,
 2345,
 69666,
 271,
 56492,
 477,
 912,
 3300,
 304,
 856,
 53101,
 11,
 323,
 4400,
 4040,
 311,
 2802,
 757,
 271,
 263,
 31284,
 11,
 358,
 3463,
 358,
 1053,
 30503,
 922,
 264,
 2697,
 323,
 1518,
 279,
 30125,
 727,
 961,
 271,
 1073,
 279,
 1917,
 13,
 1102,
 374,
 264,
 1648,
 358,
 617,
 315,
 10043,
 1022,
 279,
 87450,
 268,
 323,
 271,
 1610,
 15853,
 279,
 35855,
 13,
 43633,
 358,
 1505,
 7182,
 7982,
 44517,
 922,
 271,
 1820,
 11013,
 26,
 15716,
 433,
 374,
 264,
 41369,
 11,
 1377,
 73825,
 6841,
 304,
 856,
 13836,
 26,
 15716,
 271,
 40,
 1505,
 7182,
 4457,
 3935,
 6751,
 7251,
 985,
 1603,
 78766,
 83273,
 11,
 323,
 271,
 81088,
 709,
 279,
 14981,
 315,
 1475,
 32079,
 358,
 3449,
 26,
 323,
 5423,
 15716,
 271,
 2465,
 6409,
 981,
 636,
 1778,
 459,
 8582,
 1450,
 315,
 757,
 11,
 430,
 433,
 7612,
 264,
 3831,
 16033,
 271,
 652,
 16379,
 311,
 5471,
 757,


In [24]:
moby_out

['Call me Ishmael. Some years ago—never mind how long precisely—having\n\nlittle or no money in my purse, and nothing particular to interest me\n\non shore, I thought I would sail about a little and see the watery part\n\nof the world. It is a way I have of driving off the spleen and\n\nregulating the circulation. Whenever I find myself growing grim about\n\nthe mouth; whenever it is a damp, drizzly November in my soul; whenever\n\nI find myself involuntarily pausing before coffin warehouses, and\n\nbringing up the rear of every funeral I meet; and especially whenever\n\nmy hypos get such an upper hand of me, that it requires a strong moral\n\nprinciple to prevent me from deliberately stepping into the street, and\n\nmethodically knocking people’s hats off—then, I account it high time to\n\nget to sea as soon as I can. This is my substitute for pistol and ball.\n\nWith a philosophical flourish Cato throws himself upon his sword; I\n\nquietly take to the ship. There is nothing surprisin